In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#필요한 lib으로 한번만 하고, jupyter환경에선 다신 실행 안하셔도되고, colab에선 실행할 때마다 해주심 됩니다.
!pip install catboost
!pip install lightgbm
!pip install ngboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.5/98.5 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.2/349.2 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 10.5 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=bd8c303302f2e715b8b3a34adb10daea03c9801192c132fab103f49ac354b839
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma


In [3]:
# plt.style.available

In [4]:
# Essentials
import numpy as np
import pandas as pd
import random
import time
import gc
import os
from datetime import datetime

# Plots
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
figure(num=None, figsize=(20, 10), dpi=80, facecolor='w', edgecolor='k')
import seaborn as sns
# matplotlib 의 기본 scheme 말고
# seaborn scheme 을 세팅하여, 일일이 graph의 font size 지정할 필요 없이
# seaborn 의 font_scale 을 사용하는 것을 추천드립니다.
plt.style.use('seaborn-v0_8') # 이 위에 코드를 실행하면 여러 테마가 있으니 마음껏 바꾸셔도 됩니다.
sns.set(font_scale=2.5)
from matplotlib.pylab import rcParams
##set up the parameters
rcParams['figure.figsize'] = 80,60

# Models
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from ngboost import NGBClassifier
import lightgbm
import xgboost as xgb
import catboost
from xgboost import plot_tree

# Stats
from scipy.stats import skew, norm
from scipy.special import boxcox1p
from scipy.stats import boxcox_normmax


# Misc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, cross_val_score
from sklearn.metrics import mean_squared_error, accuracy_score

from dateutil import tz

# Ignore useless warnings
import warnings
warnings.filterwarnings(action="ignore")
pd.options.display.max_seq_items = 8000
pd.options.display.max_rows = 8000
pd.options.display.max_columns = 400
pd.set_option('display.max_colwidth', None)

<Figure size 1600x800 with 0 Axes>

In [5]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/lending_club/lending_club_2020_train.csv')
df

,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,124989905,6000.0,6000.0,6000.0,36 months,7.97%,187.94,A,A5,Teacher,2 years,MORTGAGE,45000.0,Verified,Dec-2017,Fully Paid,n,https://lendingclub.com/browse/loanDetail.action?loan_id=124989905,debt_consolidation,Debt consolidation,554xx,MN,8.67,1.0,Sep-2005,755.0,759.0,0.0,22.0,NaN,14.0,0.0,3090.0,14%,36.0,w,0.00,0.00,6718.840000,6718.84,6000.00,718.84,0.00,0.00,0.0000,Apr-2020,1835.06,NaN,Apr-2020,764.0,760.0,0.0,NaN,1.0,Joint App,100500.0,10.20,Verified,0.0,0.0,50054.0,1.0,10.0,3.0,3.0,3.0,46964.0,83.0,1.0,1.0,3090.0,64.0,22100.0,1.0,0.0,2.0,4.0,3850.0,11910.0,20.6,0.0,0.0,147.0,135.0,12.0,3.0,1.0,56.0,NaN,3.0,NaN,0.0,1.0,1.0,1.0,4.0,28.0,4.0,7.0,1.0,14.0,0.0,0.0,0.0,4.0,97.1,0.0,0.0,0.0,78611.0,50054.0,15000.0,56511.0,3926.0,555.0,559.0,Sep-2005,0.0,0.0,10.0,NaN,10.0,4.0,1.0,0.0,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,139665484,35000.0,35000.0,35000.0,60 months,14.47%,822.95,C,C2,Product Specialist,2 years,RENT,94000.0,Source Verified,Sep-2018,Current,n,https://lendingclub.com/browse/loanDetail.action?loan_id=139665484,credit_card,Credit card refinancing,085xx,NJ,17.41,0.0,Dec-2002,715.0,719.0,2.0,NaN,NaN,6.0,0.0,24640.0,35.4%,8.0,w,25993.32,25993.32,16430.860000,16430.86,9006.68,7424.18,0.00,0.00,0.0000,May-2020,822.95,Jun-2020,May-2020,754.0,750.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,58284.0,1.0,1.0,1.0,1.0,5.0,33644.0,96.0,0.0,0.0,8542.0,56.0,69600.0,1.0,0.0,2.0,1.0,9714.0,44960.0,35.4,0.0,0.0,30.0,154.0,46.0,5.0,0.0,46.0,NaN,5.0,NaN,0.0,4.0,4.0,4.0,5.0,2.0,4.0,5.0,4.0,6.0,0.0,0.0,0.0,1.0,100.0,0.0,0.0,0.0,104863.0,58284.0,69600.0,35000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
2,84253847,23200.0,23200.0,23200.0,60 months,24.99%,680.82

In [6]:
columns_list = [
    'addr_state', 'annual_inc', 'annual_inc_joint', 'application_type',
    'avg_cur_bal', 'earliest_cr_line', 'emp_length', 'emp_title',
    'fico_range_high', 'fico_range_low', 'grade', 'home_ownership',
    'inq_last_12m', 'inq_last_6mths', 'installment', 'mo_sin_old_il_acct',
    'mo_sin_old_rev_tl_op', 'purpose', 'pymnt_plan', 'sub_grade', 'title',
    'verification_status', 'sec_app_fico_range_low', 'sec_app_fico_range_high',
    'sec_app_earliest_cr_line', 'sec_app_inq_last_6mths', 'sec_app_mort_acc',
    'sec_app_open_acc', 'sec_app_open_act_il', 'sec_app_num_rev_accts',
    'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med', 'loan_status'
]

In [7]:
df = df[columns_list]

In [8]:
column_types = df.dtypes

# float와 object로 나누기
float_columns = []
object_columns = []
for column_name, dtype in column_types.items():
    if dtype == 'float':
        float_columns.append(column_name)
    elif dtype == 'object':
        object_columns.append(column_name)

print("Float 열:", float_columns)
print("Object 열:", object_columns)

Float 열: ['annual_inc', 'annual_inc_joint', 'avg_cur_bal', 'fico_range_high', 'fico_range_low', 'inq_last_12m', 'inq_last_6mths', 'installment', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'sec_app_fico_range_low', 'sec_app_fico_range_high', 'sec_app_inq_last_6mths', 'sec_app_mort_acc', 'sec_app_open_acc', 'sec_app_open_act_il', 'sec_app_num_rev_accts', 'sec_app_chargeoff_within_12_mths', 'sec_app_collections_12_mths_ex_med']
Object 열: ['addr_state', 'application_type', 'earliest_cr_line', 'emp_length', 'emp_title', 'grade', 'home_ownership', 'purpose', 'pymnt_plan', 'sub_grade', 'title', 'verification_status', 'sec_app_earliest_cr_line', 'loan_status']


In [9]:
[]

[]

In [10]:
# 타겟 변수 추가.
#float_columns.append('loan_status')

In [11]:
#df = df[float_columns]

In [12]:
# 결측치 우선 0으로 채움
#df.fillna(0, inplace = True)

In [13]:
# 결측치 있나 확인
df.isnull().sum()

addr_state                                  1
annual_inc                                  3
annual_inc_joint                      1628938
application_type                            1
avg_cur_bal                             42163
earliest_cr_line                           17
emp_length                             123085
emp_title                              158401
fico_range_high                             1
fico_range_low                              1
grade                                       1
home_ownership                              1
inq_last_12m                           519721
inq_last_6mths                             17
installment                                 1
mo_sin_old_il_acct                      93246
mo_sin_old_rev_tl_op                    42089
purpose                                     1
pymnt_plan                                  1
sub_grade                                   1
title                                   14043
verification_status               

In [14]:
stat_list = list(df['loan_status'].unique())

In [15]:
stat_list

['Fully Paid',
 'Current',
 'Charged Off',
 'Late (16-30 days)',
 'Late (31-120 days)',
 'In Grace Period',
 'Issued',
 'Does not meet the credit policy. Status:Fully Paid',
 'Does not meet the credit policy. Status:Charged Off',
 'Default',
 nan]

In [16]:
not_deliq = ['Fully Paid', 'Current', 'Does not meet the credit policy. Status:Fully Paid']

In [17]:
loan_stat = df['loan_status']
target = []
for stat in loan_stat:
    if stat in not_deliq:
        target.append(1)
    else:
        target.append(0)
print(len(target))
print(len(df))

1755295
1755295


In [18]:
df['target'] = target

In [19]:
df['target']

0          1
1          1
2          0
3          1
4          1
          ..
1755290    0
1755291    1
1755292    1
1755293    1
1755294    1
Name: target, Length: 1755295, dtype: int64

In [20]:
df = df.drop(columns = 'loan_status')

In [21]:
df

,addr_state,annual_inc,annual_inc_joint,application_type,avg_cur_bal,earliest_cr_line,emp_length,emp_title,fico_range_high,fico_range_low,grade,home_ownership,inq_last_12m,inq_last_6mths,installment,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,purpose,pymnt_plan,sub_grade,title,verification_status,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,target
0,MN,45000.0,100500.0,Joint App,3850.0,Sep-2005,2 years,Teacher,759.0,755.0,A,MORTGAGE,2.0,0.0,187.94,147.0,135.0,debt_consolidation,n,A5,Debt consolidation,Verified,555.0,559.0,Sep-2005,0.0,0.0,10.0,10.0,4.0,1.0,0.0,1
1,NJ,94000.0,NaN,Individual,9714.0,Dec-2002,2 years,Product Specialist,719.0,715.0,C,RENT,2.0,2.0,822.95,30.0,154.0,credit_card,n,C2,Credit card refinancing,Source Verified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,OK,110000.0,NaN,Individual,25264.0,May-1993,10+ years,Production Foreman,674.0,670.0,E,MORTGAGE,7.0,1.0,680.82,126.0,277.0,debt_consolidation,n,E4,Debt consolidation,Verified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,FL,35000.0,NaN,Individual,293.0,Nov-1995,10+ years,Office Manager,679.0,675.0,B,MORTGAGE,4.0,1.0,145.06,127.0,265.0,home_improvement,n,B2,Home improvement,Not Verified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,NC,29500.0,119500.0,Joint App,32845.0,Dec-2007,10+ years,Secretary,689.0,685.0,C,MORTGAGE,6.0,0.0,530.33,129.0,54.0,debt_consolidation,n,C4,Debt consolidation,Verified,655.0,659.0,Dec-2007,1.0,5.0,7.0,3.0,4.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1755290,TX,33800.0,NaN,Individual,9103.0,Aug-2001,7 years,Manager,669.0,665.0,E,MORTGAGE,NaN,0.0,133.78,151.0,36.0,debt_consolidation,n,E1,Debt consolidation,Not Verified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1755291,UT,71000.0,NaN,Individual,18466.0,Mar-1997,4 years,Systems Engineer,739.0,735.0,A,MORTGAGE,NaN,0.0,583.89,125.0,216.0,debt_consolidation,n,A3,Debt consolidation,Not Verified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1755292,TX,44000.0,NaN,Individual,20715.0,Mar-2005,9 years,ESS Support services,694.0,690.0,C,MORTGAGE,NaN,1.0,329.65,25.0,99.0,debt_consolidation,n,C1,Consolidation Loan,Verified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1755293,MD,67000.0,NaN,Individual,6918.0,Jul-2005,5 years,sales associate,739.0,735.0,A,MORTGAGE,4.0,1.0,744.90,54.0,151.0,debt_consolidation,n,A4,Debt consolidation,Source Verified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [22]:
# 우선 랜덤하게 data_split하고 돌려봄 6:2:2
df_train, df_test = train_test_split(df, test_size = .2, train_size = .8, random_state = 42)
df_train, df_validation = train_test_split(df_train, test_size = .25, train_size = 0.75, random_state = 42)

In [23]:
df_train.describe()

,annual_inc,annual_inc_joint,avg_cur_bal,fico_range_high,fico_range_low,inq_last_12m,inq_last_6mths,installment,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,target
count,1.053175e+06,7.602200e+04,1.027753e+06,1.053177e+06,1.053177e+06,741159.000000,1.053165e+06,1.053177e+06,996997.000000,1.027795e+06,71492.000000,71492.000000,71492.000000,71492.000000,71492.000000,71492.000000,71492.000000,71492.000000,71492.000000,1.053177e+06
mean,7.994756e+04,1.305247e+05,1.371792e+04,7.045541e+02,7.005539e+02,2.004582,5.547763e-01,4.520167e+02,125.851811,1.796759e+02,673.454582,677.454596,0.629049,1.583842,11.583995,3.070190,12.548677,0.038452,0.067574,8.650198e-01
std,1.028865e+05,8.558084e+04,1.667696e+04,3.378919e+01,3.378825e+01,2.355407,8.607875e-01,2.725252e+02,54.393302,9.775870e+01,44.417581,44.417635,0.983593,1.803034,6.690406,3.254882,8.200860,0.368184,0.375096,3.417025e-01
min,0.000000e+00,9.000000e+03,0.000000e+00,6.140000e+02,6.100000e+02,0.000000,0.000000e+00,1.401000e+01,0.000000,1.000000e+00,540.000000,544.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
25%,4.700000e+04,8.600000e+04,3.115000e+03,6.790000e+02,6.750000e+02,0.000000,0.000000e+00,2.535200e+02,93.000000,1.120000e+02,645.000000,649.000000,0.000000,0.000000,7.000000,1.000000,7.000000,0.000000,0.000000,1.000000e+00
50%,6.600000e+04,1.150000e+05,7.414000e+03,6.990000e+02,6.950000e+02,1.000000,0.000000e+00,3.815400e+02,130.000000,1.620000e+02,670.000000,674.000000,0.000000,1.000000,10.000000,2.000000,11.000000,0.000000,0.000000,1.000000e+00
75%,9.500000e+04,1.550000e+05,1.899800e+04,7.240000e+02,7.200000e+02,3.000000,1.000000e+00,6.031200e+02,155.000000,2.300000e+02,700.000000,704.000000,1.000000,3.000000,15.000000,4.000000,17.000000,0.000000,0.000000,1.000000e+00
max,6.100000e+07,9.706945e+06,8.311310e+05,8.500000e+02,8.450000e+02,51.000000,3.100000e+01,1.719830e+03,822.000000,9.990000e+02,845.000000,850.000000,6.000000,29.000000,82.000000,43.000000,110.000000,21.000000,19.000000,1.000000e+00


In [24]:
def show_time(diff):
   m, s = divmod(diff, 60)
   h, m = divmod(m, 60)
   s,m,h = int(round(s, 0)), int(round(m, 0)), int(round(h, 0))
   print("Execution Time: " + "{0:02d}:{1:02d}:{2:02d}".format(h, m, s))

# Takes in a classifier, calculates the training + prediction times and accuracy score, returns a model
def Train(clf, X, y, X_predict, y_predict, type='classification'):
    # Train
    start = time.time()
    model = clf.fit(X,y)
    end = time.time()
    print('Training time: ')
    show_time(end - start)
    training_times.append(end - start)

    # Predict
    start = time.time()
    if(type=='classification'):
        scores.append(accuracy_score(y_predict, model.predict(X_predict)))
    else:
        scores.append(mean_squared_error(y_test, model.predict(X_test)))
    end = time.time()
    prediction_times.append(end - start)
    print('\nPrediction time: ')
    show_time(end - start)
    return model


# Takes in model scores and plots them on a bar graph
def plot_metric(model_scores, score='Accuracy'):
    # Set figure size
    rcParams['figure.figsize'] = 20,15
    plt.bar(model_scores['Model'], height=model_scores[score])
    xlocs, xlabs = plt.xticks()
    xlocs=[i for i in range(0,17)]
    xlabs=[i for i in range(0,17)]
    if(score != 'Prediction Times'):
        for i, v in enumerate(model_scores[score]):
            plt.text(xlocs[i] - 0.25, v + 0.01, str(v))
    plt.xlabel('Model')
    plt.ylabel(score)
    plt.xticks(rotation=45)
    plt.show()


In [25]:
x_data = list(df_train.columns)
x_data.remove('target')
x_data

['addr_state',
 'annual_inc',
 'annual_inc_joint',
 'application_type',
 'avg_cur_bal',
 'earliest_cr_line',
 'emp_length',
 'emp_title',
 'fico_range_high',
 'fico_range_low',
 'grade',
 'home_ownership',
 'inq_last_12m',
 'inq_last_6mths',
 'installment',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'purpose',
 'pymnt_plan',
 'sub_grade',
 'title',
 'verification_status',
 'sec_app_fico_range_low',
 'sec_app_fico_range_high',
 'sec_app_earliest_cr_line',
 'sec_app_inq_last_6mths',
 'sec_app_mort_acc',
 'sec_app_open_acc',
 'sec_app_open_act_il',
 'sec_app_num_rev_accts',
 'sec_app_chargeoff_within_12_mths',
 'sec_app_collections_12_mths_ex_med']

In [26]:
X_train = df_train[x_data]
y_train = df_train['target']

X_test = df_validation[x_data]
y_test = df_validation['target']
# 하나하나 선택해도 되지만, 필요없는 columns은 진작에 삭제해버려도 좋을듯
# Target은 Survived

NgBoost Hyperparameter list up
https://github.com/stanfordmlgroup/ngboost/blob/51a299197ae77dff88826156edf386313201d99b/ngboost/api.py#L21

In [27]:
from sklearn.metrics import accuracy_score

In [28]:
prediction_times = []
training_times = []
scores = []
# training_times
# prediction_times

#xgboost = Train(XGBClassifier(n_estimators=50, max_depth=7), X_train, y_train, X_test, y_test)
#lgb = Train(LGBMClassifier(n_estimators=50, max_depth=7, num_leaves = 70), X_train, y_train, X_test, y_test)
cat = Train(CatBoostClassifier(n_estimators=50, verbose=False, max_depth=7), X_train, y_train, X_test, y_test)
#ngb = Train(NGBClassifier(n_estimators=50, verbose=False, natural_gradient=False), X_train, y_train, X_test, y_test)

CatBoostError: Bad value for num_feature[non_default_doc_idx=0,feature_idx=0]="CA": Cannot convert 'b'CA'' to float

In [ ]:
models = [
    #('XGBoost', xgboost),
    #('LightGBM', lgb),
    ('CatBoost', cat),
    #('NgBoost', ngb),
]

In [ ]:
model_scores = pd.DataFrame({ 'Model': [name for name, _ in models], 'Accuracy': scores })
model_scores.sort_values(by='Accuracy',ascending=False,inplace=True)
plot_metric(model_scores, score='Accuracy')